In [1]:
import numpy as np
import pandas as pd
#import GPy
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt
from scipy.linalg        import cho_factor, cho_solve
from scipy.spatial.distance import pdist, squareform
from scipy.optimize      import minimize
from scipy.linalg import svd, cholesky, eigh



In [2]:
districts = pd.read_csv("postaldistricts.txt", names=["district", "blank", "easting", "northing", "lat", "long", "NGR","grid","sources"], usecols=range(9))

In [3]:
districts

,district,blank,easting,northing,lat,long,NGR,grid,sources
0,AB1,###,383656,760468,56.735562,-2.268770,NO 836604,osgb,NPE Postcode web submission
1,AB10,###,392567,804537,57.131679,-2.124423,NJ 92545,osgb,Dracos.co.uk Postbox Importer
2,AB11,###,394533,805406,57.139507,-2.091960,NJ 94554,osgb,Dracos.co.uk Postbox Importer
3,AB12,###,393057,800339,57.093971,-2.116218,NJ 9303,osgb,Dracos.co.uk Postbox Importer
4,AB13,###,387085,802538,57.113599,-2.214879,NJ 87025,osgb,NPE Postcode web submission
...,...,...,...,...,...,...,...,...,...
3131,YO91,###,460504,453656,53.975455,-1.079029,SE 605536,osgb,NPE Postcode web submission
3132,YS22,###,493648,507760,54.456617,-0.557006,NZ 93677,osgb,NPE Postcode web submission
3133,ZE1,###,446297,1141020,60.151066,-1.167983,HU 462410,osgb,FreeThePostcode.org Importer
3134,ZE2,###,444046,1168116,60.394585,-1.202625,HU 440681,osgb,Dracos.co.uk Postbox Importer


In [4]:
prices = pd.read_csv("realestate_data_london_2024_nov_trimmed.csv")


In [5]:
prices["district"] = prices["title"].apply(lambda x: x.split(",")[-1].strip())
prices["district"] = prices["district"].apply(lambda x: x.split(" ")[-1].strip())

df = pd.merge(
    prices,
    districts,
    on='district',    # name of the column in both frames
    how='inner'         # keep all rows from df_prices
)


In [6]:
### DISTRICTS IN PRICES BUT NOT POSTCODE LOCATIONS
set(prices["district"]) - set(districts["district"])

{'E20'}

In [7]:
### VALUE COUNTS
df["propertyType"].value_counts()

propertyType
Apartment              242
Detached               152
Terraced               151
Flat                   127
House                  110
Penthouse               94
Semi-Detached           37
Town House              29
End of Terrace          20
Duplex                  10
Not Specified            8
Block of Apartments      7
Land                     6
Mews                     6
Maisonette               6
Villa                    4
Link Detached House      3
Plot                     2
Equestrian Facility      2
Character Property       1
Ground Flat              1
Name: count, dtype: int64

In [8]:
lst = ["Apartment", "Flat", "Detached", "Terraced", "House", "Penthouse", "Semi-Detached", "Town House"]
df = df[df["propertyType"].isin(lst)]

Index(['title', 'propertyType', 'sizeSqFeetMax', 'bedrooms', 'bathrooms',
       'price', 'district', 'blank', 'easting', 'northing', 'lat', 'long',
       'NGR', 'grid', 'sources'],
      dtype='object')

In [10]:
df = df[["price", "easting", "northing", "propertyType", "sizeSqFeetMax", "bedrooms", "bathrooms"]]
df = df[~df["easting"].isna()]

In [11]:
### COLUMNS WITH NANS
nan_counts = df.isna().sum()
nan_counts

price              0
easting            0
northing           0
propertyType       0
sizeSqFeetMax    130
bedrooms           4
bathrooms         22
dtype: int64

In [12]:
### MEDIA AFTER CLEANING BUT BEFORE IMUPTATION
df[["sizeSqFeetMax", "bedrooms", "bathrooms"]].median()

sizeSqFeetMax    3795.5
bedrooms            5.0
bathrooms           4.0
dtype: float64

In [13]:
dummies = pd.get_dummies(df['propertyType'], prefix='pt_', dtype=int)

# join back to your original df (and optionally drop the original column)
df2 = df.drop('propertyType', axis=1).join(dummies)

In [14]:
df2['price_float'] = (
    df2['price']
      .str.replace('£', '', regex=False)     # drop the £
      .str.replace(',', '', regex=False)     # drop the commas
      .astype(float)                         # cast to float
)


In [15]:
### IMPUTATION STEP

dummies = pd.get_dummies(df['propertyType'], prefix='pt_', dtype=int)
df2 = df.drop('propertyType', axis=1).join(dummies)
df2['price_float'] = (
    df2['price']
      .str.replace('£', '', regex=False)     # drop the £
      .str.replace(',', '', regex=False)     # drop the commas
      .astype(float)                         # cast to float
)

### FILL BLANKS WITH MEDIAN
df_f = df2.drop(columns=["price", "easting", "northing"])
for col in ["sizeSqFeetMax", "bathrooms", "bedrooms"]:
    predictors = [c for c in df2.columns if c != col]
    df2[col + "i"] = df2[col].fillna(
        df2[col].median()
    )
nan_counts = df2.isna().sum()

#USE MEDIAN TO CONSTRUCT LINEAR IMPUTATION MODELS
model_betas = {}
columns = ["sizeSqFeetMax", "bedrooms", "bathrooms"]
predictor_all = ["bedroomsi", "bathroomsi", "sizeSqFeetMaxi", "pt__Apartment", "pt__Detached", "pt__Flat", "pt__House", "pt__Penthouse", "pt__Semi-Detached", "pt__Terraced", "pt__Town House", "price_float"]

for col in columns:
    tmp = df2.dropna()
    y = tmp[col].to_numpy()
    predictors = [c for c in predictor_all if c != col + "i"]
    X = tmp[predictors].to_numpy()
    beta_hat = np.linalg.inv(X.T @ X) @ (X.T @ y)
    model_betas[col] = beta_hat
    

cols = ["sizeSqFeetMax", "bathrooms", "bedrooms"]
for col in ["sizeSqFeetMax", "bathrooms", "bedrooms"]:
    predictors = [c for c in predictor_all if c != col + "i"]
    df2[col] = df2[col].fillna(
        df2[predictors].dot(model_betas[col])
    )


In [16]:
df2[["sizeSqFeetMax", "bedrooms", "bathrooms"]].median()

sizeSqFeetMax    4045.581546
bedrooms            5.000000
bathrooms           4.000000
dtype: float64

In [17]:
X_full = df2[["easting", "northing", "sizeSqFeetMax", "bedrooms", "bathrooms","pt__Apartment", "pt__Detached", "pt__Flat", "pt__House", "pt__Penthouse", "pt__Semi-Detached", "pt__Terraced", "pt__Town House"]].to_numpy()

y = df2["price_float"].to_numpy()

In [18]:
X_full.shape

(942, 13)

In [19]:


#X_full = df2[["easting", "northing", "sizeSqFeetMax", "bedrooms", "bathrooms","pt__Apartment", "pt__Detached", "pt__Flat", "pt__House", "pt__Penthouse", "pt__Semi-Detached", "pt__Terraced", "pt__Town House"]].to_numpy()
X_full = df2[["easting", "northing", "sizeSqFeetMax", "bedrooms", "bathrooms","pt__Apartment", "pt__Detached", "pt__Flat", "pt__House", "pt__Penthouse", "pt__Semi-Detached", "pt__Terraced"]].to_numpy()

y = df2["price_float"].to_numpy()
X_full[:, :2] = X_full[:, :2] / 10000.0

#SCALE FOR STABILITY - REMEMBER TO UNSCALE
y = y / df2["price_float"].std()


n, D = X_full.shape

# Split into spatial vs covariates:
s = X_full[:, :2]        # shape (n,2)
X = X_full[:, 2:]        # shape (n, p)
X = np.hstack([np.ones((X.shape[0], 1)), X])

#scaler_X = StandardScaler()
#X = scaler_X.fit_transform(X)


p = X.shape[1]

# Ensure y is a column vector:
y = y.reshape(-1, 1)     # shape (n,1)


# -----------------------------------------------------------------------------
# 2) Covariance builder
# -----------------------------------------------------------------------------
def make_K(s, sigma_f2, ell, sigma_n2):
    """
    Build K = K_f + sigma_n2 * I,
    where (K_f)_{ij} = sigma_f2 * exp(-0.5 * ||s_i - s_j||^2 / ell^2).
    Returns (K, K_f, sq_dists).
    """
    # pairwise squared distances (n×n)
    d2 = squareform(pdist(s, 'sqeuclidean'))
    K_f = sigma_f2 * np.exp(-0.5 * d2 / ell**2)
    return K_f + sigma_n2 * np.eye(len(s)), K_f, d2


def near_spd_eig(A, tol=1e-14):
    """
    Projects symmetric A onto the nearest SPD matrix by clipping eigenvalues.
    """
    # (1) Symmetrize
    B = (A + A.T) / 2

    # (2) Eigendecompose
    eigvals, eigvecs = eigh(B)

    # (3) Clip negatives to tol
    eigvals_clipped = np.clip(eigvals, tol, None)

    # (4) Reconstruct, then re-symmetrize
    B_psd = (eigvecs * eigvals_clipped) @ eigvecs.T
    return (B_psd + B_psd.T) / 2

def make_K_higham_eig(s, sigma_f2, ell, sigma_n2, tol=1e-14):
    """
    1) Build raw K_f
    2) Project K_f to SPD
    3) Add noise sigma_n2*I
    4) Project final K to SPD
    """
    # pairwise squared distances
    d2 = squareform(pdist(s, 'sqeuclidean'))

    # raw kernel
    K_f = sigma_f2 * np.exp(-0.5 * d2 / ell**2)
    K_f = (K_f + K_f.T) / 2

    # project to SPD
    Kf_spd = near_spd_eig(K_f, tol=tol)

    # add noise
    n = K_f.shape[0]
    K = Kf_spd + sigma_n2 * np.eye(n)

    # final SPD projection
    K_spd = near_spd_eig(K, tol=tol)

    # diagnostics (optional)
    #print("min eigen(K_f after proj) =", np.min(np.linalg.eigvalsh(Kf_spd)))
    #print("min eigen(final K_spd)   =", np.min(np.linalg.eigvalsh(K_spd)))

    return K_spd, Kf_spd, d2

def make_K_fixed(s, sigma_f2, ell, sigma_n2):
    # 1) raw squared distances
    d2 = squareform(pdist(s, 'sqeuclidean'))

    # 2) raw covariance (may be rank-deficient)
    K_f = sigma_f2 * np.exp(-0.5 * d2 / ell**2)
    K_f = (K_f + K_f.T) / 2   # exact symmetry
    # 3) add noise nugget (automatically PD)
    n = K_f.shape[0]
    K = K_f + sigma_n2 * np.eye(n)
    # 4) optionally symmetrize to kill tiny round-off asymmetries
    K = (K + K.T) / 2
    return K, K_f, d2



# -----------------------------------------------------------------------------
# 3) Log-marginal likelihood + gradient (for minimization)
# -----------------------------------------------------------------------------
def neg_log_marginal_and_grad(theta, s, X, y, return_betas = False):
    """
    Given theta = [sigma_n2, sigma_f2, ell],
    returns (neg_logL, neg_grad).
    """
    sigma_n2, sigma_f2, ell = theta
    n, p = X.shape
    # Build covariance
    K, K_f, d2 = make_K_higham_eig(s, sigma_f2, ell, sigma_n2)
    #K, K_f, d2 = make_K_fixed(s, sigma_f2, ell, sigma_n2)
    # Cholesky factorization of K
    L, lower = cho_factor(K, overwrite_a=False)

    # 3a) GLS estimate of beta:  beta = (X^T K^{-1} X)^{-1} X^T K^{-1} y
    Kinv_y = cho_solve((L, lower), y)      # K^{-1} y
    Kinv_X = cho_solve((L, lower), X)      # K^{-1} X
    XtKinvX = X.T @ Kinv_X                 # p×p
    beta_hat = np.linalg.solve(XtKinvX, X.T @ Kinv_y)  # p×1
    # 3b) Residuals under the mean
    r = y - X @ beta_hat                   # n×1

    # 3c) Log-marginal likelihood
    alpha  = cho_solve((L, lower), r)      # K^{-1} r
    logdet = 2.0 * np.sum(np.log(np.diag(L)))
    logL   = -0.5 * (r.T @ alpha + logdet + (n-p) * np.log(2*np.pi))
    logL   = float(logL)                  # scalar

    # 4) Gradients w.r.t. each theta_j
    # W = alpha alpha^T - K^{-1}
    Kinv = lambda M: cho_solve((L, lower), M)
    W = alpha @ alpha.T - Kinv(np.eye(n))  # n×n

    # Derivatives of K
    dK_dsigma_n2 = np.eye(n)
    dK_dsigma_f2 = K_f / sigma_f2
    dK_dell      = K_f * (d2 / (ell**3))

    grad = 0.5 * np.array([
        np.trace(W @ dK_dsigma_n2),
        np.trace(W @ dK_dsigma_f2),
        np.trace(W @ dK_dell)
    ])  # shape (3,)

    # We minimize, so return negatives:
    print(-logL)
    if return_betas:
        return -logL, -grad, beta_hat    
    return -logL, -grad

# -----------------------------------------------------------------------------
# 5) Choose an initial guess for [sigma_n2, sigma_f2, ell]
# -----------------------------------------------------------------------------
init_sigma_n2 = 1e-1
init_sigma_f2 = np.var(y)
init_ell      = 2 * np.std(s)  # e.g. a few units in your scaled coords

theta0 = np.array([init_sigma_n2, init_sigma_f2, init_ell])

bounds = [(1e-12, None),   # sigma_n2 ≥ 1e-8
          (1e-12, None),   # sigma_f2
          (1e-8, None)]

# -----------------------------------------------------------------------------
# 6) Run the optimizer
# -----------------------------------------------------------------------------
res = minimize(
    fun    = neg_log_marginal_and_grad,
    x0     = theta0,
    args   = (s, X, y),
    jac    = True,
    bounds = bounds,
    method = 'L-BFGS-B',         # quasi-Newton; or 'Newton-CG' if you implement Hessian
    options= {'disp': True, 'gtol':1e-6}
)
print(res)
sigma_n2_opt, sigma_f2_opt, ell_opt = res.x
print("Optimized noise var:",    sigma_n2_opt)
print("Optimized signal var:",   sigma_f2_opt)
print("Optimized lengthscale:",  ell_opt)

C:\Users\daved\AppData\Local\Temp\ipykernel_4816\2710727906.py:134: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  logL   = float(logL)                  # scalar


3802.5161011723603
1270.338588824727
1270.0882501919448
1269.10253170352
1265.4312503842402
1265.0240641019832
1257.9007061234831
1257.1222799368184
1256.7963091677755
1256.7892882475362
1256.7887508303922
1256.7861644202321
1256.7803845974481
1256.7632143635435
1256.7101619953833
1254.0354281675595
1253.7814019234283
1253.264333849591
1253.2589005653058
1253.258857404536
1253.25885740057
  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 1253.25885740057
        x: [ 8.559e-01  1.000e-12  3.451e+01]
      nit: 17
      jac: [-1.933e-06  5.501e+02  7.864e-15]
     nfev: 21
     njev: 21
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
Optimized noise var: 0.855928161843549
Optimized signal var: 1e-12
Optimized lengthscale: 34.51479810749315


In [20]:
l,g,beta = neg_log_marginal_and_grad([sigma_n2_opt, sigma_f2_opt, ell_opt], s, X, y, True)
beta_unscaled = beta *  df2["price_float"].std()

### CORRECT VALUES IN BETA UNSCALED (REMEMBER TO UNSCALE)
beta, beta_unscaled

C:\Users\daved\AppData\Local\Temp\ipykernel_4816\2710727906.py:134: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  logL   = float(logL)                  # scalar


1253.25885740057


(array([[ 1.43406245e+00],
        [ 6.85552760e-06],
        [-6.01763102e-02],
        [ 1.93719523e-01],
        [-4.94452141e-01],
        [-5.09666975e-01],
        [-5.80790219e-01],
        [-2.22977828e-01],
        [-2.98040943e-01],
        [-2.90671788e-01],
        [-3.41166646e-01]]),
 array([[ 9.81037472e+06],
        [ 4.68984419e+01],
        [-4.11664185e+05],
        [ 1.32522897e+06],
        [-3.38253105e+06],
        [-3.48661524e+06],
        [-3.97316704e+06],
        [-1.52538409e+06],
        [-2.03888842e+06],
        [-1.98847627e+06],
        [-2.33390995e+06]]))

In [21]:
#y_pred = np.dot(X, beta) #+ intercept

In [22]:
###CHECK YOU GET THE SAME VALUES WITHOUT INEEFECTIVE SPATIAL PARAMETER
X_ = df2[["sizeSqFeetMax", "bedrooms", "bathrooms","pt__Apartment", "pt__Detached", "pt__Flat", "pt__House", "pt__Penthouse", "pt__Semi-Detached", "pt__Terraced"]].to_numpy()
X_ = np.hstack([np.ones((X_.shape[0], 1)), X_])

y_ = df2["price_float"].to_numpy()#/ df2["price_float"].std()

XtX = X_.T @ X_              # shape (p, p)
XtX_inv = np.linalg.inv(XtX)
Xty = X_.T @ y_              # shape (p,)
beta_hat = XtX_inv @ Xty   
beta_hat

array([ 9.81037472e+06,  4.68984419e+01, -4.11664185e+05,  1.32522897e+06,
       -3.38253105e+06, -3.48661524e+06, -3.97316704e+06, -1.52538409e+06,
       -2.03888842e+06, -1.98847627e+06, -2.33390995e+06])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from scipy.linalg        import cho_solve, cho_factor

# --- 1) Your trained params and data (replace these) -----------------------
sigma_n2, sigma_f2, ell = sigma_n2_opt, sigma_f2_opt, ell_opt
s_train = s                        # (n,2) spatial coords from your data
X_train = X                        # (n,p) covariates
y_train = y.flatten()              # (n,)
# your fitted regression weights:
_, _, beta_hat = neg_log_marginal_and_grad(
    [sigma_n2, sigma_f2, ell], s, X, y, return_betas=True)

# build and factor the training covariance
K = make_K(s_train, sigma_f2, ell, sigma_n2)[0]
L, lower = cho_factor(K, check_finite=False)

# --- 2) Fix non‐spatial covariates to their means --------------------------
x_ref = np.mean(X_train, axis=0)   # (p,)

# --- 3) Build grid over spatial domain -------------------------------------
nx = ny = 100
x1 = np.linspace(s_train[:,0].min(), s_train[:,0].max(), nx)
x2 = np.linspace(s_train[:,1].min(), s_train[:,1].max(), ny)
X1, X2 = np.meshgrid(x1, x2)
grid_pts = np.column_stack([X1.ravel(), X2.ravel()])  # (nx*ny, 2)

# replicate reference covariates for each grid point
X_ref = np.tile(x_ref, (grid_pts.shape[0], 1))        # (nx*ny, p)

# --- 4) Compute cross‐covariance and GP mean -------------------------------
# k(s_train, grid)
d2 = cdist(s_train, grid_pts, 'sqeuclidean')
K_star = sigma_f2 * np.exp(-0.5 * d2 / ell**2)       # (n, nx*ny)

# residuals (y - X beta)
resid = y_train - X_train.dot(beta_hat).flatten()

# predictive mean of the zero‐mean GP part
alpha = cho_solve((L, lower), resid)                 # (n,)
mu_gp = K_star.T.dot(alpha)                          # (nx*ny,)

# add back the linear mean X_* β
mu_lin = X_ref.dot(beta_hat).flatten()               # (nx*ny,)
mu = mu_gp + mu_lin

# reshape to grid
MU = mu.reshape(ny, nx)

# --- 5) Plot ---------------------------------------------------------------
plt.figure(figsize=(6,5))
pcm = plt.pcolormesh(X1, X2, MU, shading='auto')
plt.scatter(s_train[:,0], s_train[:,1], c=y_train, edgecolor='k', cmap='viridis')
plt.colorbar(pcm, label='Predicted response')
plt.xlabel('Easting (×10000)')
plt.ylabel('Northing (×10000)')
#plt.title('GP spatial effect + covariate mean')
plt.show()
